# Mexican Covid

At the end of 2019, the world saw the born of the Sars-cov-2 virus which genereate a disease called COVID19 (COrona VIrus disease). The high risk that we face with this virus is that the sympthons are related to a typical flu but generate a high damage un the lungs decreasing the oxygen level. The disminution of oxygen generate a high demand of intensive beds.

From WHO files

According to the WHO the incubation period for COVID-19, is on average 5-6 days, but can be extended to 14 days. However there some evidence that shows that exist different types of transmition, such as symptomatic, pre-symptomatic and a symptomatic.

The typical transmition is the symptomatic, this ocurrs when a person shown COVID symptomatic and can spread the virus conciently or inconciently respiratory droplest over thier enviroment. However, some cases has shown that a person can transmit COVID-19 without symptoms, (3 or 2 days) before present the typical symptoms. Finally there are some concern about the asyptomatic transmission according to the WHO: Asymptomatic transmission refers to transmission of the virus from a person, who does not develop symptoms. There are few reports of laboratory-confirmed cases who are truly asymptomatic, and to date, there has been no documente asymptomatic transmission. This does not exclude the possibility that it may occur. Asymptomatic cases have been reported as part of contact tracing efforts in some countries. Source https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200402-sitrep-73-covid-19.pdf?sfvrsn=5ae25bc7_2#:~:text=An%20asymptomatic%20laboratory%2Dconfirmed,more%20information%20becomes%20available.


The goal of this notebook is listed below:
* Understand the evolution of the covi19 in Mexico, there are a rate of 10% of deads though a visualization analysis


Key elements

* Incubation period can be expresed a GAMMA.
* Is most common the transmision after show symptoms.
* How we can modeled (explain pre-symtomatic transmition)
* how we can modeled asyntomatic transmition


In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib 
import glob

Using matplotlib backend: MacOSX


In [2]:
import glob
strpath = '../src/'
m = []
for filename in glob.glob(strpath+"*"):
    print(filename)
    date = filename[7:9] +'-' + filename[9:11] +'-' +filename[11:13]
    df = pd.read_csv(filename, encoding = "ISO-8859-1",dtype =str)
    df.FECHA_ACTUALIZACION = pd.to_datetime(df['FECHA_ACTUALIZACION'],)
    df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)
    df.FECHA_SINTOMAS = pd.to_datetime(df['FECHA_SINTOMAS'],)
    df['deltaSintomas']=df.FECHA_ACTUALIZACION-df.FECHA_SINTOMAS 
    df['deltaSintomas'] = df.deltaSintomas.dt.days
    df.loc[df.FECHA_DEF=='9999-99-99', 'FECHA_DEF'] = '2050-12-31'
    df.FECHA_DEF = pd.to_datetime(df['FECHA_DEF'],)
    m.append([date,df[(df.deltaSintomas<14)& (df.RESULTADO=='1')]['ID_REGISTRO'].count()
          ,df[df.RESULTADO=='1']['ID_REGISTRO'].count()
          ,df[df.FECHA_DEF!='2050-12-31']['ID_REGISTRO'].count()
          ,df[df.RESULTADO=='3']['ID_REGISTRO'].count()
          ,df.ID_REGISTRO.nunique()]
    )
df = pd.DataFrame(m)
df.columns = ['date', 'activos', 'positivos', 'muertes', 'sospechosos', 'muestra']


../src/200610COVID19MEXICO.csv
../src/200413COVID19MEXICO.csv
../src/200514COVID19MEXICO.csv
../src/200426COVID19MEXICO.csv
../src/200418COVID19MEXICO.csv
../src/200521COVID19MEXICO.csv
../src/200604COVID19MEXICO.csv
../src/200504COVID19MEXICO.csv
../src/200525COVID19MEXICO.csv
../src/200621COVID19MEXICO.csv
../src/200422COVID19MEXICO.csv
../src/200531COVID19MEXICO.csv
../src/200510COVID19MEXICO.csv
../src/200417COVID19MEXICO.csv
../src/200429COVID19MEXICO.csv
../src/200614COVID19MEXICO.csv
../src/200611COVID19MEXICO.csv
../src/200412COVID19MEXICO.csv
../src/200515COVID19MEXICO.csv
../src/200501COVID19MEXICO.csv
../src/200605COVID19MEXICO.csv
../src/200427COVID19MEXICO.csv
../src/200419COVID19MEXICO.csv
../src/200520COVID19MEXICO.csv
../src/200524COVID19MEXICO.csv
../src/200620COVID19MEXICO.csv
../src/200423COVID19MEXICO.csv
../src/200601COVID19MEXICO.csv
../src/200505COVID19MEXICO.csv
../src/200511COVID19MEXICO.csv
../src/200416COVID19MEXICO.csv
../src/200428COVID19MEXICO.csv
../src/2

In [3]:
df = df.sort_values(by='date')
df = df.reset_index(drop=True)
df

,date,activos,positivos,muertes,sospechosos,muestra
0,20-04-12,255,4661,616,8697,36594
1,20-04-13,413,5014,704,9341,38670
2,20-04-14,627,5399,855,10792,41593
3,20-04-15,918,5847,937,11717,44436
4,20-04-16,1283,6297,1012,12340,47235
...,...,...,...,...,...,...
66,20-06-17,22209,159793,26561,59076,441670
67,20-06-18,23528,165455,27369,59778,453481
68,20-06-19,23653,170485,28206,62245,465867
69,20-06-20,23567,175202,28666,60621,473952


In [4]:
df = pd.read_csv('../src/200621COVID19MEXICO.csv', encoding = "ISO-8859-1",dtype =str)

In [5]:
df.FECHA_ACTUALIZACION = pd.to_datetime(df['FECHA_ACTUALIZACION'],)
df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)
df.FECHA_SINTOMAS = pd.to_datetime(df['FECHA_SINTOMAS'],)
df['deltaSintomas']=df.FECHA_ACTUALIZACION-df.FECHA_SINTOMAS
df['deltaSintomas'] = df.deltaSintomas.dt.days
df.loc[df.FECHA_DEF=='9999-99-99', 'FECHA_DEF'] = '2050-12-31'
df.FECHA_DEF = pd.to_datetime(df['FECHA_DEF'],)
m=[]
m.append([df[(df.deltaSintomas<14)& (df.RESULTADO=='1')]['ID_REGISTRO'].count(),
          df[df.RESULTADO=='1']['ID_REGISTRO'].count()
          ,df[df.FECHA_DEF>'2050-12-31']['ID_REGISTRO'].count()
          ,df[df.RESULTADO=='3']['ID_REGISTRO'].count()
          ,df.ID_REGISTRO.nunique()])

In [6]:
m.append([date,df[(df.deltaSintomas<14)& (df.RESULTADO==1)]['ID_REGISTRO'].count(),
          df[df.RESULTADO==1]['ID_REGISTRO'].count()
          ,df[df.FECHA_DEF!='2050-12-31']['ID_REGISTRO'].count()
          ,df[df.RESULTADO==3]['ID_REGISTRO'].count()
          ,df.ID_REGISTRO.nunique()])

In [7]:
df[(df.deltaSintomas<14)& (df.RESULTADO=='1')]['ID_REGISTRO']

0         1aa78e
1         195210
11        020346
116       0e3e2f
171       01b888
           ...  
180476    04ef87
180510    10184b
180523    0f90a8
180524    1c7833
180525    0952a8
Name: ID_REGISTRO, Length: 24225, dtype: object

In [8]:
len(df[(df.FECHA_DEF<'2020-06-22')])-len(df[(df.FECHA_DEF<'2020-06-22')&(df.RESULTADO!='3')])

1892

In [14]:
len(df[(df.deltaSintomas<14)& (df.RESULTADO=='1')])

24225

In [21]:
len(df[(df.deltaSintomas<14)& (df.RESULTADO=='3')])

35875

In [22]:
len(df[(df.FECHA_DEF<'2020-12-31')&(df.RESULTADO=='3')])

1892

In [23]:
len(df[(df.FECHA_DEF<'2020-12-31')&(df.RESULTADO=='2')])

6035